In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import gc
import sys
sys.path.append('./tools')

from load_data import load_data, load_split_data, load_set_data
from input_distortion import input_distortion,pretraining_input_distortion
from visualization import print_training_frames
from copy import copy

from PretrainingMAE import PretrainingMAE
from MAE import MAE

In [ ]:
# Load data for training
train_seq, val_seq, test_seq = load_split_data()

data_train = load_set_data(train_seq)
data_val = load_set_data(val_seq)

In [ ]:
# Define models for Pretraining
pretraining =  PretrainingMAE(data_train,data_val)
del data_train, data_val

In [ ]:
# Pretraing RGB Channels independently
pretraining.pretrain_red_channel()
pretraining.pretrain_green_channel()
pretraining.pretrain_blue_channel()

In [ ]:
# Pretraing Inverse-Depth Channel
pretraining.pretrain_depth_channel()

In [ ]:
# Pretraing Semantic Channels independently
pretraining.pretrain_gnd_channel()
pretraining.pretrain_obj_channel()
pretraining.pretrain_bld_channel()
pretraining.pretrain_veg_channel()
pretraining.pretrain_sky_channel()

In [ ]:
# Pretraing Semantic AutoEncoder
pretraining.pretrain_shared_semantics()

In [2]:
#del pretraining

# Load data for training and testing
train_seq, val_seq, test_seq = load_split_data()

data_train = load_set_data(train_seq)
data_val = load_set_data(val_seq)
data_test = load_set_data(test_seq)

In [3]:
# Define Full Model
mae = MAE(data_train,data_val,data_test)
del data_train, data_val, data_test

In [ ]:
# Train MultiModal AutoEncoder
mae.train_model()

In [4]:
# Evaluate full MAE
mae.evaluate(run='20171016-131619')

INFO:tensorflow:Restoring parameters from models/full/20171016-131619/fullmodel.ckpt


INFO:tensorflow:Restoring parameters from models/full/20171016-131619/fullmodel.ckpt


('Size of Test Set:', 18894)
('Error (RMS):', 10.32117176574792)
('Error (Relative Error):', 0.33716038165288492)


In [ ]:
print len(data_val[1])

In [ ]:
b1 = np.reshape(data_val[1][10]['xcr1']/255.,(60,18)).T
b2 = np.reshape(data_val[1][10]['xcr2']/255.,(60,18)).T

%matplotlib inline

fig,axes = plt.subplots(2,1)
axes[0].imshow(b1)
axes[0].set_title('Left')

axes[1].imshow(b2)
axes[1].set_title('Right')

plt.show()

In [ ]:
#a = np.reshape(pretraining.imr_val[0],(60,18))
#print a[1]
#np.reshape(pretraining.imr_val)
print test_seq